In [ ]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.runnables import RunnableConfig
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore


# import tools from parent directory, set parent dir for jupyter notebook only test
import os
import sys
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# find tools in parent dir
if os.path.isdir(os.path.join(parent_dir, 'tools')):
    # Add parent directory to sys.path if found
    sys.path.append(parent_dir)

# it is needed for py files
from tools.rag import rag_tool

from dotenv import load_dotenv
load_dotenv()

QDRANT_URL = os.getenv("QDRANT_URL")

# api key in env variable QDRANT__SERVICE__API_KEY
client_qd = QdrantClient(url=QDRANT_URL)

In [10]:
# for deploy models use from vllm server
emb_model_name = '../../../../models/Qwen3-Embedding-0.6B'
embeddings = HuggingFaceEmbeddings(model_name=emb_model_name)

In [ ]:
# create classes for Qdrant filter
class Qdruls(BaseModel):
    'Rules for filtering Qdrant vector store.'
    key: str = Field(..., description="The field to filter on.")
    match: dict = Field(..., description="The match condition for the filter.")


class Qdfilter(BaseModel):
    """Filter for Qdrant vector store."""
    must: List[Qdruls] = Field(
        ...,
        description="List of rules to apply for filtering the vector store.",
    )

In [ ]:
collection_name = "recall_memory"

vectorstore_recall = rag_tool(
    client_qd=client_qd,
    collection_name=collection_name,
    embeddings=embeddings
)


def get_user_thread_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    thread_id = config["configurable"].get("thread_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")
    if thread_id is None:
        raise ValueError("Thread ID needs to be provided to save a memory.")

    return user_id, thread_id



def save_recall_memory(memory: str, config: RunnableConfig, filter: Qdfilter=None) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id, thread_id = get_user_thread_id(config)
    document = Document(
        page_content=memory, metadata={"user_id": user_id, 'thread_id': thread_id}
    )
    vectorstore_recall.add_documents([document])
    return memory



def search_recall_memories(query: str, config: RunnableConfig, filter: Qdfilter=None) -> List[str]:
    """Search for relevant memories."""
    user_id, thread_id = get_user_thread_id(config)

    # # Qdrant filter: match payload field "user_id" == user_id and "thread_id" == thread_id
    # qdrant_filter = {
    #     "must": [
    #         {
    #             "key": "user_id",
    #             "match": {"value": user_id},
    #         },
    #         {
    #             "key": "thread_id",
    #             "match": {"value": thread_id},
    #         },
    #     ]
    # }

    documents = vectorstore_recall.similarity_search(
        query,
        k=3,
        filter=filter,  # structured filter required by QdrantVectorStore
    )
    return [doc.page_content for doc in documents]


